In [7]:
import pandasai as pai
from dotenv import load_dotenv
from pandasai_openai import OpenAI

load_dotenv()

llm = OpenAI()  # reads OPENAI_API_KEY by default
pai.config.set({"llm": llm})

In [17]:
import pandas as pd
import pandasai as pai
from pandasai import Agent

# 1. Prepare your DataFrame however you like:
data = {"country": ["US", "UK", "FR", "DE", "IT"], "sales": [5000, 3200, 2900, 4100, 2300]}

data2 = {
    "country": ["us", "uk", "fr", "de", "it"],
    "population": [3000000, 6000000, 2100000, 3500000, 2000000],
}

# Prepare DataFrames however you like
df_sales = pai.DataFrame(data)
df_pop = pai.DataFrame(data2)

df_sales.to_csv("sales.csv", index=False)
df_pop.to_csv("pop.csv", index=False)

In [18]:
# Prepare DataFrames however you like
df_sales = pai.DataFrame(data)
df_pop = pai.DataFrame(data2)

# Initialize a new Agent for this session
agent = Agent(dfs=[df_sales, df_pop], memory_size=10)

In [19]:
agent.chat("can you do some analysis on which country is good for my business", output_type='string')

StringResponse(type='string', value='The best country for your business based on sales per capita is US with a sales per capita of 0.00.')

In [55]:
agent.follow_up("what question did I ask you last time?", output_type="string")

StringResponse(type='string', value='Top 5 countries by sales are: US: 5000, DE: 4100, UK: 3200, FR: 2900, IT: 2300.')

In [23]:
agent.follow_up("what is the value of apple I asked you last time?")

NumberResponse(type='number', value=0.0011804761904761906)

In [44]:
import pandas as pd
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

# 1. Load your DataFrame
df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")

# 2. Instantiate the Chat model
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# 3. Create the pandas-agent
agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    agent_type="zero-shot-react-description",
    verbose=True,
    allow_dangerous_code=True,
)

# 4. Run a query
print(agent.run("How many passengers survived?"))

/tmp/ipykernel_45181/3669747754.py:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(agent.run("How many passengers survived?"))




> Entering new AgentExecutor chain...
Thought: To find the number of passengers who survived, I need to count the number of rows where the 'Survived' column is 1.

Action: python_repl_ast

Action Input: df['Survived'].sum()
342Final Answer: 342 passengers survived.

> Finished chain.
342 passengers survived.


In [46]:
agent.__dict__

{'name': None,
 'memory': None,
 'callbacks': None,
 'verbose': True,
 'tags': None,
 'metadata': None,
 'callback_manager': None,
 'agent': RunnableAgent(runnable=RunnableAssign(mapper={
   agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
 })
 | PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'df_head': '|    |   PassengerId |   Survived |   Pclass | Name                                                | Sex    |   Age |   SibSp |   Parch | Ticket           |    Fare | Cabin   | Embarked   |\n|---:|--------------:|-----------:|---------:|:----------------------------------------------------|:-------|------:|--------:|--------:|:-----------------|--------:|:--------|:-----------|\n|  0 |             1 |          0 |        3 | Braund, Mr. Owen Harris                             | male   |    22 |       1 |       0 | A/5 21171        |  7.25   | nan     | S          |\n|  1 |             2 |          1